In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('cleaned_laptop_data1.csv')
#df.drop(columns=['Unnamed: 0','Gpu'],axis='columns',inplace=True)

In [3]:
df.head()

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,ppi,Cpu brand,HDD,SSD,Gpu brand,os
0,Apple,Ultrabook,8,1.37,71378.6832,0,1,226.983005,Intel Core i5,0,128,Intel,Mac
1,Apple,Ultrabook,8,1.34,47895.5232,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,HP,Notebook,8,1.86,30636.0000,0,0,141.211998,Intel Core i5,0,256,Intel,Others/No OS/Linux
3,Apple,Ultrabook,16,1.83,135195.3360,0,1,220.534624,Intel Core i7,0,512,AMD,Mac
4,Apple,Ultrabook,8,1.37,96095.8080,0,1,226.983005,Intel Core i5,0,256,Intel,Mac


In [4]:
new_col = {
    'Cpu brand':'Cpu_brand',
    'Gpu brand':'Gpu_brand'

}
df.rename(columns=new_col,inplace=True)

In [5]:
Cpu brand

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3291472702.py, line 1)

In [6]:
df.sample()

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,ppi,Cpu_brand,HDD,SSD,Gpu_brand,os
586,HP,Gaming,16,3.78,79813.44,0,1,127.335675,Intel Core i7,1000,256,Nvidia,Windows


In [6]:
#df.to_csv("cleaned_laptop_data2.csv",index=False,header=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1302 entries, 0 to 1301
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1302 non-null   object 
 1   TypeName     1302 non-null   object 
 2   Ram          1302 non-null   int64  
 3   Weight       1302 non-null   float64
 4   Price        1302 non-null   float64
 5   Touchscreen  1302 non-null   int64  
 6   IPS          1302 non-null   int64  
 7   ppi          1302 non-null   float64
 8   Cpu_brand    1302 non-null   object 
 9   HDD          1302 non-null   int64  
 10  SSD          1302 non-null   int64  
 11  Gpu_brand    1302 non-null   object 
 12  os           1302 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 132.4+ KB


In [11]:
for col in df.columns:
    if col == 'Price':
        pass
    else:
        print(col)
        print(df[col].unique())
        print("*"*100)

Company
['Apple' 'HP' 'Acer' 'Asus' 'Dell' 'Lenovo' 'Chuwi' 'MSI' 'Microsoft'
 'Toshiba' 'Huawei' 'Xiaomi' 'Vero' 'Razer' 'Mediacom' 'Samsung' 'Google'
 'Fujitsu' 'LG']
****************************************************************************************************
TypeName
['Ultrabook' 'Notebook' 'Netbook' 'Gaming' '2 in 1 Convertible'
 'Workstation']
****************************************************************************************************
Ram
[ 8 16  4  2 12  6 32 24 64]
****************************************************************************************************
Weight
[1.37  1.34  1.86  1.83  2.1   2.04  1.3   1.6   2.2   0.92  1.22  0.98
 2.5   1.62  1.91  2.3   1.35  1.88  1.89  1.65  2.71  1.2   1.44  2.8
 2.    2.65  2.77  3.2   0.69  1.49  2.4   2.13  2.43  1.7   1.4   1.8
 1.9   3.    1.252 2.7   2.02  1.63  1.96  1.21  2.45  1.25  1.5   2.62
 1.38  1.58  1.85  1.23  1.26  2.16  2.36  2.05  1.32  1.75  0.97  2.9
 2.56  1.48  1.74  1.1   1.56  2.03  1.05 

In [7]:
df.drop(columns=['ppi'],axis=1,inplace=True)

In [8]:
X = df.drop(columns=['Price'])
y = pd.DataFrame(np.log(df['Price']))

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=2)

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, 183 to 1192
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1106 non-null   object 
 1   TypeName     1106 non-null   object 
 2   Ram          1106 non-null   int64  
 3   Weight       1106 non-null   float64
 4   Touchscreen  1106 non-null   int64  
 5   IPS          1106 non-null   int64  
 6   Cpu_brand    1106 non-null   object 
 7   HDD          1106 non-null   int64  
 8   SSD          1106 non-null   int64  
 9   Gpu_brand    1106 non-null   object 
 10  os           1106 non-null   object 
dtypes: float64(1), int64(5), object(5)
memory usage: 103.7+ KB


In [11]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(drop='first',sparse_output=False),[0,1,6,9,10])
],remainder='passthrough')

step2 = RandomForestRegressor(n_estimators=100,
                            random_state=3,
                            max_samples=0.5,
                            max_features=0.75,
                            max_depth=15)

pipe = Pipeline([
    ('step1',step1),
    #('step2',step2)
])

X_train_encoded = step1.fit_transform(X_train)
X_test_encoded = step1.transform(X_test)

step2.fit(X_train_encoded,y_train)

y_pred = step2.predict(X_test_encoded)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

d:\propredict\pp\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


R2 score 0.8879698349726288
MAE 0.15756874019613865


In [14]:
X_train.sample()

,Company,TypeName,Ram,Weight,Touchscreen,IPS,Cpu_brand,HDD,SSD,Gpu_brand,os
1190,Lenovo,Gaming,8,2.4,0,1,Intel Core i5,1000,0,Nvidia,Windows


In [13]:
step2.predict('MSI','Gaming',16,2.43,0,1,'Intel Core i7',1000,256,'Nvidia'	,'Windows')

TypeError: ForestRegressor.predict() takes 2 positional arguments but 12 were given

In [18]:

import pickle


In [20]:
pickle.dump(step1,open('prep.pkl','wb'))

In [19]:

pickle.dump(step2,open("mod.pkl",'wb'))

In [16]:
with open('artifacts/laptop/prep.pkl','rb') as f:
    f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/laptop/prep.pkl'